In [2]:
import pandas as pd
ratings = pd.read_csv('../ml-latest-small/ratings.csv')

In [3]:
movies = pd.read_csv("../ml-latest-small/movies.csv")

In [4]:
user_movie_matrix = pd.merge(ratings, movies, on='movieId')

In [5]:
print(user_movie_matrix.head())

   userId  movieId  rating  timestamp                        title  \
0       1        1     4.0  964982703             Toy Story (1995)   
1       1        3     4.0  964981247      Grumpier Old Men (1995)   
2       1        6     4.0  964982224                  Heat (1995)   
3       1       47     5.0  964983815  Seven (a.k.a. Se7en) (1995)   
4       1       50     5.0  964982931   Usual Suspects, The (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                               Comedy|Romance  
2                        Action|Crime|Thriller  
3                             Mystery|Thriller  
4                       Crime|Mystery|Thriller  


In [6]:
user_movie_matrix_pivot = user_movie_matrix.pivot(index='userId', columns='movieId', values='rating').fillna(0)
print(user_movie_matrix_pivot.head())

movieId  1       2       3       4       5       6       7       8       \
userId                                                                    
1           4.0     0.0     4.0     0.0     0.0     4.0     0.0     0.0   
2           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
5           4.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

movieId  9       10      ...  193565  193567  193571  193573  193579  193581  \
userId                   ...                                                   
1           0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
2           0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
3           0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
4           0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0

In [7]:
# Count seen and unseen movies

# already_rated = user_movie_matrix_pivot.loc[utente][user_movie_matrix_pivot.loc[utente] > 0].index.tolist()
# print(len(already_rated))

# not_seen = user_movie_matrix_pivot.columns[~user_movie_matrix_pivot.columns.isin(already_rated)].tolist()
# print(len(not_seen))

140


In [9]:
from scipy.sparse import csr_matrix

sparse_user_movie_matrix = csr_matrix(user_movie_matrix_pivot.values)
print(sparse_user_movie_matrix)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 100836 stored elements and shape (610, 9724)>
  Coords	Values
  (0, 0)	4.0
  (0, 2)	4.0
  (0, 5)	4.0
  (0, 43)	5.0
  (0, 46)	5.0
  (0, 62)	3.0
  (0, 89)	5.0
  (0, 97)	4.0
  (0, 124)	5.0
  (0, 130)	5.0
  (0, 136)	5.0
  (0, 184)	5.0
  (0, 190)	3.0
  (0, 197)	5.0
  (0, 201)	4.0
  (0, 224)	5.0
  (0, 257)	3.0
  (0, 275)	3.0
  (0, 291)	5.0
  (0, 307)	4.0
  (0, 314)	4.0
  (0, 320)	5.0
  (0, 325)	4.0
  (0, 367)	3.0
  (0, 384)	4.0
  :	:
  (609, 9238)	5.0
  (609, 9246)	4.5
  (609, 9256)	4.0
  (609, 9268)	5.0
  (609, 9274)	3.5
  (609, 9279)	3.5
  (609, 9282)	3.0
  (609, 9288)	3.0
  (609, 9304)	3.0
  (609, 9307)	2.5
  (609, 9312)	4.5
  (609, 9317)	3.0
  (609, 9324)	3.0
  (609, 9339)	4.0
  (609, 9341)	4.0
  (609, 9348)	3.5
  (609, 9371)	3.5
  (609, 9372)	3.5
  (609, 9374)	5.0
  (609, 9415)	4.0
  (609, 9416)	4.0
  (609, 9443)	5.0
  (609, 9444)	5.0
  (609, 9445)	5.0
  (609, 9485)	3.0


In [10]:
import numpy as np
from scipy.sparse.linalg import svds

# SVD decomposition
latent_features = 20
U, sigma, Vt = svds(sparse_user_movie_matrix, k=latent_features)
sigma = np.diag(sigma)

print('U.shape', U.shape)
print('sigma.shape', sigma.shape)
print('Vt.shape', Vt.shape)

U.shape (610, 20)
sigma.shape (20, 20)
Vt.shape (20, 9724)


In [12]:
user_movie_matrix_hat = np.dot(np.dot(U, sigma), Vt)

In [13]:
# Undo the pivot done earlier to obtain a dataframe with user_id, movie_id and rating as columns
user_movie_matrix_hat = pd.DataFrame(user_movie_matrix_hat, columns=user_movie_matrix_pivot.columns, index=user_movie_matrix_pivot.index)
print(user_movie_matrix_hat.head())

movieId    1         2         3         4         5         6         7       \
userId                                                                          
1        2.290336  1.460203  1.033507 -0.061334 -0.002275  1.243261  0.029650   
2        0.038570  0.015272  0.016968  0.002944  0.019201 -0.005821 -0.025436   
3       -0.015220  0.049067  0.047202 -0.004936 -0.035349  0.052758 -0.012911   
4        2.238621  0.060011  0.039384  0.066455  0.221806  0.487591  0.318594   
5        1.358363  0.970071  0.340939  0.121053  0.479936  0.628346  0.504583   

movieId    8         9         10      ...    193565    193567    193571  \
userId                                 ...                                 
1        0.056161  0.036220  1.442856  ... -0.008584 -0.007358 -0.009810   
2        0.000918  0.010531 -0.117149  ...  0.010662  0.009139  0.012186   
3        0.010422 -0.002532 -0.014094  ...  0.000029  0.000025  0.000033   
4       -0.057422  0.016371  0.234273  ...  0.002029

In [16]:
user_movie_matrix_hat_unpivot = user_movie_matrix_hat.reset_index().melt(id_vars='userId', var_name='movieId', value_name='rating')
user_movie_matrix_hat_unpivot = user_movie_matrix_hat_unpivot.merge(user_movie_matrix[['userId', 'movieId', 'timestamp', 'title', 'genres']], on=['userId', 'movieId'], how='left')
user_movie_matrix_hat_unpivot = user_movie_matrix_hat_unpivot[['userId', 'movieId', 'rating', 'timestamp', 'title', 'genres']]


In [19]:
print(user_movie_matrix_hat_unpivot.head())
print(user_movie_matrix_hat_unpivot.info())

   userId movieId    rating    timestamp             title  \
0       1       1  2.290336  964982703.0  Toy Story (1995)   
1       2       1  0.038570          NaN               NaN   
2       3       1 -0.015220          NaN               NaN   
3       4       1  2.238621          NaN               NaN   
4       5       1  1.358363  847434962.0  Toy Story (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                                          NaN  
2                                          NaN  
3                                          NaN  
4  Adventure|Animation|Children|Comedy|Fantasy  


In [ ]:
user = 1
user_hat = user_movie_matrix_hat_unpivot[user_movie_matrix_hat_unpivot['userId'] == user]
user_hat.head()

In [ ]:
# Ground Truth
user_gt = user_movie_matrix[user_movie_matrix['userId'] == user]
user_gt = user_gt[['movieId', 'rating']]
user_gt.head()

In [ ]:
# Confront ground truth with svd reconstructed values
user_hat_seen = user_hat.dropna()
user_hat_seen = pd.merge(user_hat_seen, user_gt, on='movieId')
user_hat_seen.head()

In [ ]:
#pip install scikit-learn

In [ ]:
from sklearn.metrics import root_mean_squared_error

rmse = root_mean_squared_error(user_hat_seen['rating_y'], user_hat_seen['rating_x'])
print(rmse)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(metricName='rmse',
                                predictionCol='prediction',
                                labelCol='rating')

rmse = evaluator.evaluate(predicted_ratings)
print(rmse)

In [ ]:
# Movies not seen by the user
user_hat = user_hat[user_hat['timestamp'].isna()]
user_hat = user_hat.sort_values('rating', ascending=False).head(10)
user_hat = user_hat.drop(['title', 'genres', 'timestamp'], axis=1)
user_hat = pd.merge(user_hat, movies, on='movieId')
display(user_hat)
user_hat.info()

In [ ]:
recs = user_hat['title'].tolist()
print(recs)

In [ ]:
# Connect to Neo4j
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
username = "neo4j"
password = "testtest"

driver = GraphDatabase.driver(uri, auth=(username, password)

In [ ]:
upload_test = True

In [ ]:
# Delete old test
if upload_test:
    def delete_test(tx, userId):
        tx.run("MATCH (u:User {userId: $userId})-[r:SVD_RECOMMENDED]->(m:Movie) DELETE r", userId=userId)

    with driver.session() as session:
        for movieId in s_t:           
            session.execute_write(delete_test, user)

In [ ]:
# Upload the test
if upload_test:
    def create_recommendations(tx, userId, recs):
            for rec in recs:
                tx.run("MATCH (u:User {userId: $userId}), (m:Movie {title: $title})"
                    "MERGE (u)-[:SVD_RECOMMENDED]->(m)",
                    userId=userId, title=rec)

    with driver.session() as session:
        session.execute_write(create_recommendations, user, recs)

In [ ]:
driver.close()

In [ ]:
exit()